In [8]:
B=1400
GLM_name='Poisson'  # or your model
n = 700  # Main sample size
p = 10   # Number of features
r = int(n**0.8)  # Size of each sub-sample
maintain=1
folder='resultspart2'

In [9]:
import pandas as pd
import  numpy as np
import torch


def sigmoid(x):
    return 1/(1+np.exp(-x))



def f_2(x):
    return 2.0 * torch.tanh((1.5*x[:,0] + 0.6*((x[:,2]**2) - 1) + 0.4*x[:,3]*torch.tanh(x[:,4]) + 0.15*torch.sin(x[:,5])) / 2.5)



# 1) Read CSVs (no header)
df_bf1 = pd.read_csv(f"{folder}/{GLM_name}fBf2n{n}p{p}B{B}r{r}.csv", header=None)
df_sd  = pd.read_csv(f"{folder}/{GLM_name}sdf2nn{n}p{p}B{B}r{r}.csv", header=None)
df_sdcrt  = pd.read_csv(f"{folder}/{GLM_name}sdcrtf2nn{n}p{p}B{B}r{r}.csv", header=None)
xtest = torch.load(f"{folder}/xtest{p}.pt")
# 2) Convert to NumPy arrays, then to torch.Tensor
#    (you can specify dtype=torch.float32 if needed)
tensor_bf1 = torch.tensor(df_bf1.values, dtype=torch.float32)
tensor_sd  = torch.tensor(df_sd.values,  dtype=torch.float32)
tensor_sdcrt  = torch.tensor(df_sdcrt.values,  dtype=torch.float32)

truef = torch.tensor(f_2(xtest), dtype=torch.float32)

if GLM_name=="Bernoulli":
    def activate(x):
        return 1/(1+np.exp(-x))
    
    true_f=truef.numpy()

    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)
    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    stdcrt = np.nan_to_num(stdcrt, nan=0.0)
    std0=np.std(estimated_f,axis=0)
    std1=np.mean(stdcrt,axis=0)
    true_f=true_f[index]
   
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    cover0=((estimated_f0 >= estimated_f-1.96*stdcrt) & (estimated_f0 <= estimated_f+1.96*stdcrt))
    cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    
    # within_confidence_interval = ((true_f >= estimated_f-1.96*std) & (true_f <= estimated_f+1.96*std))
elif GLM_name=="Poisson":
    def activate(x):
        return np.exp(x)
    true_f=np.log(np.log(1+np.exp(truef.numpy())))
    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)

    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    stdcrt = np.nan_to_num(stdcrt, nan=0.0)
    std0=np.std(estimated_f,axis=0)
    std1=np.mean(std,axis=0)
    true_f=true_f[index]
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    cover0=((estimated_f0 >= estimated_f-1.96*stdcrt) & (estimated_f0 <= estimated_f+1.96*stdcrt))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    
    # cover0=((true_f >= estimated_f-1.96*std0) & (true_f <= estimated_f+1.96*std0))
    # cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    # true_lambda=np.exp(true_f)
    # estimated_lambda=np.exp(estimated_f)
    
    # within_confidence_interval = ((true_f >= estimated_f-1.96*std) & (true_f <= estimated_f+1.96*std))

    # # Step 2: Count the number of times true_probability is within the confidence interval
    # count_within_interval = within_confidence_interval.sum()
else: 
    def activate(x):
        return 1/(1+np.exp(-x))
    true_f=truef.numpy()

    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)
    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    std0=np.std(estimated_f,axis=0)
    std1=np.mean(stdcrt,axis=0)
    true_f=true_f[index]
   
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    cover0=((estimated_f0 >= estimated_f-1.96*stdcrt) & (estimated_f0 <= estimated_f+1.96*stdcrt))
    cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))


print("biasf:",np.mean(estimated_f0-true_f),"(sd):",np.std(estimated_f-true_f) )
print("MAEf:",np.mean(np.abs(estimated_f-true_f)) ,"(sd):", np.std(np.abs(estimated_f-true_f)) )
print("biaspsi:",np.mean((activate(estimated_f)-activate(true_f))),"sd:",np.std((activate(estimated_f)-activate(true_f))) )
print("MAEpsi:",np.mean(np.abs(activate(estimated_f)-activate(true_f))),"SD:",np.std(np.abs(activate(estimated_f)-activate(true_f))) )
print('The empirical std:',np.mean(std0))
print('SE:',np.mean(std))
print('SE_c:',np.mean(stdcrt))
print("coverprob",np.mean(np.mean(cover,axis=0)))
print("coverprob_unbias",np.mean(np.mean(cover0,axis=0)))
print("AIL:",AIL)
print("AILsd:",AILsd)



biasf: -0.21413319 (sd): 0.3692153
MAEf: 0.3248709 (sd): 0.27682453
biaspsi: -0.14236112 sd: 0.27327946
MAEpsi: 0.21474409 SD: 0.22098261
The empirical std: 0.30186117
SE: 0.41006806
SE_c: 0.3068544
coverprob 0.8795
coverprob_unbias 0.946125
AIL: 0.77661616
AILsd: 0.5216155


/tmp/ipykernel_1769825/4179720956.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  truef = torch.tensor(f_2(xtest), dtype=torch.float32)
